In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')



import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


import warnings

warnings.filterwarnings('ignore')


In [13]:
data=pd.read_csv(r'C:\Users\Pakistan\Downloads\emails.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Pakistan\\Downloads\\emails.csv'